In [1]:
!python -m venv myenv

In [2]:
!myenv\Scripts\activate

In [ ]:
!pip install langchain_community

### Load model 

In [4]:
from langchain_community.llms import Ollama

In [5]:
llm = Ollama(model="llama2")

### Read transaction data

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("transactions_2022_2023.csv")
unique_transactions = df["Name / Description"].unique()

In [ ]:
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

# Parsing 30 transactions at a time
index_list = list(hop(0, len(unique_transactions), 30))
index_list

### Categorize transaction data

In [ ]:
# Output validation
from pydantic import BaseModel, field_validator
from typing import List

# Validate response format - check if it actually contains hyphen ("-")
class ResponseChecks(BaseModel):
    data: List[str]

    @field_validator("data")
    def check(cls, value):
        for item in value:
            if len(item) > 0:
                assert "-" in item, "String does not contain hyphen."

In [9]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    # Keep only the lines in between blank lines (removing the explaination lines at the beginning and end of the response)
    blank_indexes = [index for index in range(len(response)) if response[index] == '']
    if len(blank_indexes) == 1:
        response = response[(blank_indexes[0] + 1):]
    else:
        response = response[(blank_indexes[0] + 1) : blank_indexes[1]]
    
    # Validate response
    print(response)
    ResponseChecks(data = response)
    
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [ ]:
categories_df_all = pd.DataFrame()
max_tries = 7

for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    # Try and validate output, if it fails, try again for max_tries=7 times
    for j in range(1, max_tries):
        try:
            categories_df = categorize_transactions(transaction_names, llm)
            categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)
            
        except:
            if j < max_tries:
                continue
            else:
                raise Exception(f"Cannot categorise transactions indexes {i} to {i+1}.")
        break

In [ ]:
categories_df_all

In [18]:
categories_df_all.to_csv("categories_df_all.csv", index=False)

In [ ]:
unique_categories = categories_df_all["Category"].unique()
unique_categories

In [ ]:
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [ ]:
# Remove the numbering from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

In [22]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("transactions_2022_2023.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df

,Date,Name / Description,Expense/Income,Amount (EUR),Transaction vs category,Transaction,Category
0,2023-12-30,Belastingdienst,Expense,9.96,1. Belastingdienst - Taxes,Belastingdienst,Taxes
1,2023-12-30,Tesco Breda,Expense,17.53,2. Tesco Breda - Groceries,Tesco Breda,Groceries
2,2023-12-30,Monthly Appartment Rent,Expense,451.00,3. Monthly Appartment Rent - Housing,Monthly Appartment Rent,Housing
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46,4. Vishandel Sier Amsterdam - Food Delivery,Vishandel Sier Amsterdam,Food and Drinks
4,2023-12-29,Selling Paintings,Income,13.63,5. Selling Paintings - Art & Design,Selling Paintings,Art & Design
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19,6. Spotify Ab By Adyen - Entertainment,Spotify Ab By Adyen,Entertainment
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08,7. Tk Maxx Amsterdam Da - Shopping,Tk Maxx Amsterdam Da,Shopping
7,2023-12-22,Consulting,Income,541.57,8. Consulting - Professional Services,Consulting,Services
8,2023-12-22,Aidsfonds,Expense,10.70,9. Aidsfonds - Donations,Aidsfonds,Donations
9,2023-12-20,Consulting,Income,2641.93,8. Consulting - Professional Services,Consulting,Services


In [23]:
df.to_csv("transactions_2022_2023_categorized.csv", index=False)